In [7]:
import pandas as pd
import requests
import time



In [8]:


# Load the data into a DataFrame
df = pd.read_csv('../data/splited/location_crop.csv')

In [9]:


# Define the API endpoint and the API key
url = "https://api-bdc.io/graphql/reverse-geocoding?key=bdc_07958ebe48f74e0dad1f87f56fc5aed7"

# Set up the request headers
headers = {
    'Content-Type': 'application/json'
}

In [10]:



# Function to get location data from the API
def get_location_data(latitude, longitude):
    query = """
    {
      locationData(latitude: %f, longitude: %f, locale: "en") {
        locality {
          continent
          continentCode
          principalSubdivision
          city
          localityName
          postcode
          fips {
            state
            county
            countySubdivision
            place
          }
        }
        country {
          name
          isoAlpha2
          isoAlpha3
        }
      }
    }
    """ % (latitude, longitude)

    payload = {
        "query": query
    }

    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.text)
        return None




In [11]:
# Iterate over the DataFrame and get location data for each row
results = []

for index, row in df.iterrows():
    listing_link = row['listing_link']
    latitude = row['latitude']
    longitude = row['longitude']
    result = get_location_data(latitude, longitude)
    if result:
        result['listing_link'] = listing_link
        result['latitude'] = latitude
        result['longitude'] = longitude
        results.append(result)
    time.sleep(1)  # Sleep to avoid hitting rate limits

# Extract relevant data and store in a new DataFrame
location_data = []

for result in results:
    if 'data' in result and 'locationData' in result['data']:
        locality = result['data']['locationData']['locality']
        country = result['data']['locationData']['country']
        location_data.append({
            'listing_link': result['listing_link'],
            'latitude': result['latitude'],
            'longitude': result['longitude'],
            'continent': locality['continent'],
            'continentCode': locality['continentCode'],
            'principalSubdivision': locality['principalSubdivision'],
            'city': locality['city'],
            'localityName': locality['localityName'],
            'postcode': locality['postcode'],
            'fips': locality['fips'],
            'country_name': country['name'],
            'country_isoAlpha2': country['isoAlpha2'],
            'country_isoAlpha3': country['isoAlpha3']
        })

location_df = pd.DataFrame(location_data)

Request failed with status code 400
{"errors":[{"message":"Argument \u0027latitude\u0027 has invalid value. Expected type \u0027Float\u0027, found nan.","locations":[{"line":3,"column":20}],"extensions":{"code":"ARGUMENTS_OF_CORRECT_TYPE","codes":["ARGUMENTS_OF_CORRECT_TYPE"],"number":"5.6.1"}},{"message":"Argument \u0027longitude\u0027 has invalid value. Expected type \u0027Float\u0027, found nan.","locations":[{"line":3,"column":35}],"extensions":{"code":"ARGUMENTS_OF_CORRECT_TYPE","codes":["ARGUMENTS_OF_CORRECT_TYPE"],"number":"5.6.1"}}],"extensions":{"elapsedMs":11.8089,"processedBy":"43.204.238.189"}}
Request failed with status code 400
{"errors":[{"message":"Argument \u0027latitude\u0027 has invalid value. Expected type \u0027Float\u0027, found nan.","locations":[{"line":3,"column":20}],"extensions":{"code":"ARGUMENTS_OF_CORRECT_TYPE","codes":["ARGUMENTS_OF_CORRECT_TYPE"],"number":"5.6.1"}},{"message":"Argument \u0027longitude\u0027 has invalid value. Expected type \u0027Float\u0

Cluster 3:
- apple fruit
- orange fruit
- banana fruit

Cluster 1:
- carrot vegetable
- spinach vegetable
- tomato vegetable

Cluster 2:
- dog animal
- cat animal
- elephant animal



In [12]:
# Print the resulting DataFrame
print(location_df)

# Save the resulting DataFrame to a CSV file
location_df.to_csv('location_data.csv', index=False)

                                           listing_link   latitude  longitude  \
0     https://www.airbnb.com/rooms/31699739?adults=1...  24.778140  90.368330   
1     https://www.airbnb.com/rooms/94052072774264244...  24.698140  90.433410   
2     https://www.airbnb.com/rooms/10463077937269931...  24.757100  90.401200   
3     https://www.airbnb.com/rooms/93979221326458612...  24.757807  90.394817   
4     https://www.airbnb.com/rooms/93979244890044911...  24.757011  90.394424   
...                                                 ...        ...        ...   
1564  https://www.airbnb.com/rooms/10257503846395791...  22.778333  89.724451   
1565  https://www.airbnb.com/rooms/52017590?adults=1...  22.590000  89.848400   
1566  https://www.airbnb.com/rooms/11301829800703733...  22.702143  90.371497   
1567  https://www.airbnb.com/rooms/53844727?adults=1...  21.818050  90.121490   
1568  https://www.airbnb.com/rooms/51131644?adults=1...  22.721640  90.368900   

     continent continentCod